In [8]:
using CSV

In [2]:
COMPETITION_PATH = "../data/competition.csv"
MODEL_INPUT_PATH = "../data/model_input.csv"
TRAINING_DATA_PATH = "../data/all_games_7_players.csv"
TREE_PATH = "../processed/OCTs/all_players_1.json"

"../processed/OCTs/all_players_1.json"

In [5]:
# Given a trained tree lnr, and a matrix of , return the mean and std of each node populated by the injected features
function get_nodes_mean_std(lnr, train_X, train_y; only_leafs = false)
    if only_leafs
        leafs = get_leafs(lnr)
        nodes_elements = IAI.apply_nodes(lnr, train_X)[leafs]
    else
        nodes_elements = IAI.apply_nodes(lnr, train_X)
    end
    nb_nodes = length(nodes_elements)
    stds = zeros(length(nodes_elements))
    means = zeros(length(nodes_elements))
    for i=1:nb_nodes
        stds[i] = std(Vector(train_y)[nodes_elements[i]])    
        means[i] = mean(Vector(train_y)[nodes_elements[i]])
    end
    means, stds
end;

function get_leafs(lnr)
    num_nodes = IAI.get_num_nodes(lnr)
    leafs = []
    for i=1:num_nodes
        if IAI.is_leaf(lnr, i)
            push!(leafs, i)
        end
    end
    leafs
end;

function get_leaf_pred_mean_std(lnr, new_X, train_X, train_y)
    nodes_means, nodes_stds = get_nodes_mean_std(lnr, train_X, train_y);
    predictions = IAI.predict(lnr, new_X)  
    assigned_leafs = IAI.apply(lnr, new_X)
    leafs_means = nodes_means[assigned_leafs]
    leafs_stds = nodes_stds[assigned_leafs]
    DataFrame(assigned_leaf = assigned_leafs,
        prediciton = predictions,
        leaf_mean = leafs_means,
        leaf_std = leafs_stds
    )
end;

In [9]:
lnr = IAI.read_json(TREE_PATH)

,player_id_p1,fgm_seas_avg_p1,fgm_seas_avg_p2,fgm_seas_avg_p3,fgm_seas_avg_p4
,String,Float64,Float64,Float64,Float64
1,robinna01,6.06757,4.65672,7.8481,5.54878
2,bryanko01,9.7561,4.84416,4.39726,7.30864
3,flynnjo01,0.0,7.08861,2.42424,6.35714
4,wallara01,4.57576,7.98701,6.31646,2.97826
5,howardw01,7.08861,2.42424,6.35714,6.05063
6,flynnjo01,0.0,9.7,3.90123,0.0
7,wadedw01,10.8101,7.0,3.63889,5.47887
8,westbru01,5.31707,8.93243,5.08333,7.54545
9,jamesle01,9.74074,6.5942,5.46341,3.13115


In [ ]:
test_X = CSV.read(MODEL_INPUT_PATH)

In [ ]:
train = CSV.read(TRAINING_DATA_PATH)

In [10]:
train_X = train[:,1:end-1]
train_y = train[:,end]

163865-element Array{Float64,1}:
 69.2
 65.5
 51.9
 45.2
 30.9
 24.1
 66.1
 56.3
 60.3
 54.9
 47.3
 59.7
 21.8
  ⋮  
 44.6
 29.4
 24.0
 18.6
 15.7
 16.6
 42.7
 16.2
 19.8
 22.4
 37.5
 39.8

In [12]:
pred_mean_std = get_leaf_pred_mean_std(lnr, test_X, train_X, train_y)

ArgumentError: ArgumentError: Column 1 (:player_id_p1) of `X` should have name `:fgm_seas_avg_p1`